# 3. Properties, getter and setter

In section 1, we have seen that instance parameter can be private and only be accessible via getter and setter. But this will increase the development workload.

Note the private parameter in python is **not really private**. We can still access it. For more detail, please visit section 1 Classes_and_instances.



## 3.1 Why we need getter and setter?

Let's consider below example.

In [ ]:
class P1:
    def __init__(self,x):
        self.x=x

With above class, there is no encapsulation. Let's update it to version 2

In [ ]:
class P2:
    def __init__(self,x):
        self.__x=x
    def get_x(self):
        return self.__x
    def set_x(self,x):
        self.__x=x

It's better, but we can see there is a code duplication between set_x and __init__. So we can update it to version 3

In [ ]:
class P3:
    def __init__(self,x):
        self.set_x(x)
    def get_x(self):
        return self.__x
    def set_x(self,x):
        self.__x=x

Now the code is much better and easy to maintain. Imagine, now we need to change the logic of the initialization of x. From now on the attribute x can only have values between 0 and 1000. If a value larger than 1000, x should be set to 1000. Correspondingly, x should be set to 0, if the value is less than 0.

### 3.1.1 Why P1 is bad?
With P1, there is no way to do this. Because user can modify the value of x as they want. Check below example

In [1]:
class P1:
    def __init__(self,x):
        if x <0:
           self.x=0
        elif x >1000:
            self.x=1000
        else:
            self.x=x

In [2]:
p1=P1(2000)
print(p1.x)

1000


So far so good, the logic is respected. But check below example. As user can assign value to x directly, the logic is not respected anymore. So version 1 is the bad way to implement a class

In [3]:
p1.x=2000
print(p1.x)

2000


### 3.1.2 Why p2 is bad?

With version 2, as user can only assign value to x via setter, so the logic will be respected. But we duplicate code, if any changes occur, we need to modify two code block.


### 3.1.3 P3 is a good start

We can see all the logic that can modify x is located in set_x(), it's much easier to maintain the code. And x is only accessible via getter and setter, so we have the total control over the x logic.

In [4]:
class P3:
    def __init__(self,x):
        self.set_x(x)
    def get_x(self):
        return self.__x
    def set_x(self,x):
        if x <0:
           self.__x=0
        elif x >1000:
            self.__x=1000
        else:
            self.__x=x

In [5]:
p3=P3(2000)
print(p3.get_x())

p3.set_x(2100)
print(p3.get_x())

1000
1000


## 3.2 Properties decorator

Is there a way in python to allow us to apply getter and setter on public attributes? Imagine the following code (version 1) has been used in many place.
```python
p1 = P1(42)
p1.x = 1001
p1.x
```

If we change the version 1 signature. Many code will be broken. To avoid this, We can use property decorator


In [8]:
class P4:
    def __init__(self,x):
        self.x=x
    @property
    def x(self):
        return self.__x
    @x.setter
    def x(self, x):
        if x < 0:
            self.__x = 0
        elif x > 1000:
            self.__x = 1000
        else:
            self.__x = x

In [9]:
p4=P4(2000)
print(p4.x)

1000


In [10]:
p4.x = 2900
print(p4.x)

1000


In the above class example, we define a getter function with **def x(self):** and use the **@property** decorator on top of it. So when we call `print(p4.x)`, it's this getter function thar are called.


We defined a setter function with **def x(self, x):** and use the "@x.setter" decorator. If the function had been called "f", we would have to decorate it with "@f.setter".


Two things are noteworthy: We just put the code line "self.x = x" in the __init__ method and the property method x is used to check the limits of the values. The second interesting thing is that we wrote "two" methods with the same name and a different number of parameters "def x(self)" and "def x(self,x)". We have learned in a previous chapter of our course that this is not possible. It works here due to the decorating

## 3.3 Internals of the property decorator

The property decorator has a complex mechanism to implement the getter and setter method. Here we just show you a tip of the iceberg.

Below example is equivalent of the above decorator example, but we don't use any decorator. Several Notes on below example:
 - we use function property to register the getter and setter of x. **x = property(__get_x, __set_x)**
 - getter and setter are privates. So the only way to use them is to use **property (p5.x)**

In [11]:
class P5:
    def __init__(self, x):
        self.__set_x(x)
    # private methods
    def __get_x(self):
        return self.__x

    def __set_x(self, x):
        if x < 0:
            self.__x = 0
        elif x > 1000:
            self.__x = 1000
        else:
            self.__x = x
    # register getter and setter function for a variable
    x = property(__get_x, __set_x)

In [12]:
p5=P5(2100)
print(p5.x)

p5.x=3900
print(p5.x)

1000
1000


## 3.4 Property can apply on anything.

We just see how to use property on instance variables. In fact, it can be used on any function. Check below example

In [13]:
class Robot:
    def __init__(self, name, build_year, lk = 0.5, lp = 0.5 ):
        self.name = name
        self.build_year = build_year
        self.__potential_physical = lk
        self.__potential_psychic = lp

    @property
    def condition(self):
        s = self.__potential_physical + self.__potential_psychic
        if s <= -1:
           return "I feel miserable!"
        elif s <= 0:
           return "I feel bad!"
        elif s <= 0.5:
           return "Could be worse!"
        elif s <= 1:
           return "Seems to be okay!"
        else:
           return "Great!"

We just set property on top of the function condition, and condition is not the name of any instance variable. So property is just a way to register a getter function, then you can call it. It does not need to be a getter of a variable, it can be a getter of any state of the object.

In [14]:
x = Robot("Marvin", 1979, 0.2, 0.4 )
y = Robot("Caliban", 1993, -0.4, 0.3)

print(x.condition)
print(y.condition)

Seems to be okay!
I feel bad!


## 3.5 Conclusion

Let's assume that we are designing a new class and we pondering about an instance or class attribute "OurAtt", which we need for the design of our class. We have to observe the following issues:

- Will the value of "OurAtt" be needed by the possible users of our class?
- If not, we can or should make it a private attribute.
- If it has to be accessed, we make it accessible as a public attribute
- We will define it as a private attribute with the corresponding property, if and only if we have to do some checks or transformation of the data. (As an example, you can have a look again at our class P, where the attribute has to be in the interval between 0 and 1000, which is ensured by the property "x")
- Alternatively, you could use a getter and a setter, but using a property is the Pythonic way to deal with it!

And the **@property** and **@fun.setter** can help you to register the getter and setter with a unique name.